## Churn Problem

##### Introduction

---


The purpose of this notebook is to design a model which tells us for a given dataset whether a customer has churned or not. The definition is to be valid for all kinds of non-contractual servies whether it be fashion retail, electronics retail, etc. 
Also, for this purpose the given notebook will also define the number of months from the last transaction date , after which we can tell if a person has churned or not.

##### Challenges

---

1) Defining the time period after which a customer can be said to have churned is in itself a great challenge. Since, in this regard churn has to be a duration which is both useful and meaningful for the company to which we are selling the model to. Like coming out with a churn figure of say more than a year , is pretty useless.

There are various features one can use to define churn: 1) like using the various transaction dates, 






### **Importing libraries & loading the required data **



>  First of all we import the necessary packages into our notebook.







In [ ]:
#Importing necessary packages.
%matplotlib inline 
import pandas as pd #Data Manipulation package
import numpy as np #Linear Algebra package
import matplotlib.pyplot as plt
import seaborn as sns # Seaborn gives us better aesthetics than basic matplotlib in our plots.
from sklearn.preprocessing import Imputer # Imputation package of scikit-learn.

In [ ]:
"""!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive"""

In [ ]:
# Changing the directory
"""import os
os.chdir("drive/ML")"""
!pwd

In [ ]:
!ls /content

There are two files for us to consider on which we have to define churn & train and test our model.
sales.csv contains the sales data, whereas cap_reg contains the customer personal details & details relating to loyalty points & other schemes.

#### For reading the two files we are using the pandas library, and we require the encoding parameter in sales.csv. And the dtype & encoding parameter for cap_reg  

Encoding is required for pandas to read the obscure characters from other languages such as Arabic, as is the case here. For other regions encoding has to be supplied in the same manner.
Link to standard python encodings is given below:
[List of Standard Python encodings.](https://docs.python.org/3/library/codecs.html#standard-encodings)

Dtype is the type of the columns which can be float64, int , str ,etc but , when a column contains more than one type of values say both string and int , then in such cases of mixed values object is used. Giving the dtype=object value makes all the columns dtype=object


In [ ]:
#Reading the data
sales = pd.read_csv("sales.csv", encoding= "ISO-8859-1")
cap_reg = pd.read_csv("cap_reg.csv", encoding="ISO-8859-1", dtype=object)

In [ ]:
sales.head()

In [ ]:
#No. of instances
len(sales) 

The below tuple shows that there are customers with only negative transactions also. This will occur in rare cases, 
#### This type of case pops up maybe, since the customer didn't give his phone number when he bought the item, but gave his phone number on return. Such people won't have customerid when they are buying items.

In [ ]:
sales['custaccount'].nunique(), sales.loc[sales['sales']> 0, 'custaccount'].nunique()

#### For this model to work there should be a transdate column, named as transdate
Changing transdate column to datetime format

In [ ]:
sales['transdate'] = pd.to_datetime(sales['transdate'], yearfirst=True, errors='coerce')

### Each receiptid can have multiple rows, since in 1 transaction there can be multiple items purchased.

 Note 1 receiptid can have several rows, since a transaction with several different items is listed in different rows.
Below, we can see multiple items purchased by a single customer in a single receipt id. As can be seen receiptid ending with 392 has 7 items in one single transaction

In [ ]:
sales.sort_values('receiptid') #Not inplace. just sorting here for displaying information.

#### Creating a new dataframe df, with only buys/no-replacements, and columns : custaccount & transdate.

### Here, I am aiming on defining churn i.e, no. of months after which , if a customer doesn't make a transaction, we can say that he has churned or we have lost that customer. So for defining churn I am using the interpurchase times between the different transaction dates and then plotting a cumulative plot.

In [ ]:
df = (sales[sales['sales'] > 0][['custaccount', 'transdate']])

In [ ]:
len(df)

### Deleting rows with same custaccount and transdate, so that duplicates get dropped

 Removing duplicates due to multiple items purchased in one transaction, and sorting by custaccount & transdate, so , as to get transdates in ascending order grouped by custaccount.

In [ ]:
df = df.sort_values(by = ['custaccount', 'transdate']).drop_duplicates()

In [ ]:
len(df['custaccount'].unique())

#### Sorted by custaccounts and their different transaction dates by ascending order.

In [ ]:
df.head(20) 

#### Grouping by custaccount and taking the different transactions differences.
#### NaT for the customers who had made only single transactions, since groupby diff doesn't work for them, since after groupby they are left with single row & don't have a previous row.
pandas.DataFrame.diff : Calculates the difference of a DataFrame element compared with another element in the DataFrame (default is the element in the same column of the previous row). - From documentation




In [ ]:
df['diff'] =  df.groupby('custaccount')['transdate'].diff()

In [ ]:
df.head(20)

#### Dropping customers with single transactions by dropping the rows with NaT 
For datetime64[ns] types, NaT represents missing values. This is a pseudo-native sentinel value that can be represented by NumPy in a singular dtype (datetime64[ns]). pandas objects provide intercompatibility between NaT and NaN. - From Documentation (Pandas)

Also dropping transdate column as it isn't required now.

---



In [ ]:
df= df.dropna(how='any').drop('transdate',axis=1)

In [ ]:
df.head(20)

Setting index as custaccount

In [ ]:
df.set_index('custaccount', inplace=True)

In [ ]:
df.head()

#### Converting the timedelta values to integer by dividing by a single day timedelta and then using astype to convert to 'int' dtype.

In [ ]:
df = (df/np.timedelta64(1, 'D')).astype('int')

In [ ]:
df.head()

#### pandas.DataFrame.values : Return a Numpy representation of the DataFrame. - From documentation

In [ ]:
df.values

In [ ]:
# Number of bins
num_bin = df.max().values

**Using np.histogram to get the frequencies of the different interpurchase time like [1, 3, 5], which would represent 0 days - 1 interpurchase time, 1 days - 3 interpurchase time, 2 days - 5 interpurchase times, etc. and then taking the cumulative sum of the values numpy arrays thus, generated and then plotting the cumulative graph.**

In [ ]:
values, base = np.histogram(df.values, bins=np.arange(num_bin))
#evaluate the cumulative
cumulative = np.cumsum(values)
# plot the cumulative function
plt.plot(base[:-1], cumulative, c='blue')

plt.show()

#### Normally we would take the churn period to be the number of days at which the slope is almost zero (gradient), but here as we can see the slope becomes zero after 400 days and defining churn duration as more than a year's time is redundant. On examination of the graph in this case we can see it looks like pareto cdf plot and thus, we can apply the pareto 80:20 rule, which says that 80% of the wealth of the people is distributed between 20% of the population.

Another approach would be to use boxplots and to exclude the outliers, as I've described latter.
![alt text](https://www.safaribooksonline.com/library/view/think-stats/9781449308117/httpatomoreillycomsourceoreillyimages863940.png)

#### churn_days defined by using the 80:20 rule

In [ ]:
# Use of Pareto 80:20 rule to solve for churn_days, boxplots could also have been used to remove outliers using concept of interquartile range
churn_days = (((cumulative[::-1].cumsum()/cumulative.sum()) * 100) >= 80).sum()

In [ ]:
churn_days

#### Using boxplots the values beyond 75 % percentile + 1.5 * IQR are outliers and can be removed. Similarly, points beyond 25% percentile - 1.5 * IQR are counted as outliers and can be removed.
We could have taken the value of iq75 (75 percentile) + 1.5 * IQR as the churn period.

In [ ]:
sns.set(style="whitegrid")
_ = sns.boxplot(x = df['diff'], notch=True, whis=1.5)


# Points outside the vertical line that is the whisker are outliers , whis=1.5 , also default parameter.

In [ ]:
# Outlier removal using concept of interquartile range.
#iq75, iq25 = np.percentile(df['diff'], [75, 25])
#iqr = iq75 - iq25

In [ ]:
# Days of churn calculated using interquartile range concept
#iq75 + iqr * 1.5                               

#### Violin plot to show the same concept of outliers and the density where the most points are situated/located.

In [ ]:
ax = sns.violinplot(df['diff'], scale="count", inner="box")

### Now converting the number of days after which a customer churns into months, respectively.

#### If days are more than or equal to 15 then convert to 1 month, if less than 15 then floor to 0. Using lambda function and numpy.ceil & numpy.floor.

In [ ]:
# This is the months function to be used as below
months = lambda x : np.ceil(x) if (x - int(x))>= 0.5 else np.floor(x)

In [ ]:
churn_months = months(churn_days/30).astype('int')

#### Number of months after which a customer can be told to have churned after his last transaction.

In [ ]:
churn_months

In [ ]:
df.head()

In [ ]:
# Creating a new dataframe df, with only buys/no-replacements, and columns : custaccount & transdate.
df1 = (sales[sales['sales'] > 0][['custaccount', 'transdate']])

In [ ]:
df1.head()

#### dropping duplicates on custaccount and keeping the last row for that custaccount. The last row would have the latest transaction date for the customers.

In [ ]:
df1 = pd.DataFrame(df1.groupby('custaccount')['transdate'].agg('max')).reset_index()

In [ ]:
df1.head()

#### Adding number of days since last transaction to the dataframe df2 by using pandas.merge.

In [ ]:
df2 = pd.merge(df1, pd.DataFrame(pd.to_datetime('today') - df1.groupby('custaccount' )['transdate'].agg('max')).reset_index().rename(columns={'transdate': 'nod'}))

In [ ]:
df2.head()

In [ ]:
df2['nod'] = (df2['nod']/np.timedelta64(1, 'D')).astype('int')

### For dataframe using dataframe.apply to scale no. of days since last transaction to months

In [ ]:
df2['no.months'] = ((df2['nod']/30).apply(lambda x : np.ceil(x) if (x - int(x)) >= 0.5 else np.floor(x))).astype('int')

In [ ]:
df2['churn'] = None

In [ ]:
df2['churn'] = df2["no.months"] >= churn_months

In [ ]:
df2.head()

In [ ]:
df2['transdate'].max()

Dropping columns transdate and nod, and then sorting by column: no.months

In [ ]:
df2.drop(columns=['transdate', 'nod']).sort_values(by='no.months')

In [ ]:
df2['churn'].unique()

### Cap_reg dropping columns

This dataframe contains personal information about the customer, such as his country, Nationality , gender , email address, points, lifetime purchases, etc.

Below we get the column names for the dataframe cap_reg

#### Using the .info method we can see the number of non-null objects in each column as well as the memory usage. This dataframe alone uses 1.4 Gigabytes RAM.

In [ ]:
df2.head()

In [ ]:
df2.drop(['transdate', 'nod', 'no.months'], axis=1, inplace=True)

In [ ]:
df2.head()

In [ ]:
len(df2)

In [ ]:
sales.head()

In [ ]:
sales1 = pd.DataFrame(sales.groupby('custaccount')['sales'].agg('sum'))

In [ ]:
#pd.merge(sales1 , sales[sales['sales']> 0], indicator=True)
sales1.head()

In [ ]:
sales['department'].unique()

#### Adding quantities of different items bought Ladies, Lingerie & Nightwear, Men , Home, etc. using groupby. By running a for loop over the unique items in the department column of sales.

In [ ]:
for i in range(len(sales['department'].unique())):
    k = sales['department'].unique()[i]
    sales1[k] = sales.where(sales['department'] == k).groupby('custaccount')['department'].agg('size')

In [ ]:
sales1.head()

In [ ]:
sales1.reset_index(inplace=True) # Resetting the index

#### Replacing all NaNs will value of 0

In [ ]:
sales1 = sales1.where(~sales1.isnull(), 0)

In [ ]:
sales1.head(20)

In [ ]:
sales1 =pd.merge(sales1, df2, on='custaccount', how='left')

In [ ]:
sales1.head()

In [ ]:
len(sales1.loc[sales1['churn'].isnull(),['custaccount', 'sales']])

In [ ]:
len(sales1)

In [ ]:
sales1['churn'].unique()

In [ ]:
sales1.head()

### The next cell shows us sales value where the model has NaN values for churn column

In [ ]:
sales1.loc[sales1['churn'].isnull(), ['custaccount', 'sales', 'churn']] # As can be seen no, all customers account  with negative sales are found here

### Deleting the rows with churn value of nan


Churn value of NaN is for those customers who didn't have positive transactions and only had negative transactions.

In [ ]:
 sales1.drop(index= sales1[sales1['churn'].isnull()].index, inplace=True)

In [ ]:
sales1['churn'].unique()

In [ ]:
sales1[sales1['sales'] < 0]

In [ ]:
sales['store'].nunique()

In [ ]:
#No. of different stores the person has shopped in.
sales.groupby('custaccount')['store'].agg(['nunique', 'size'])

### Sale dataframe completed 

In [ ]:
sales1.head()

In [ ]:
sales1[sales1['churn'] == False]

### Removing attributes from cap_reg with more than 90% missing values

In [ ]:
cap_reg.head()

#### Predefined Drop; columns not required : These columns are supposed to not be pulled when importing data using sequel server, if any column isn't required ; it is best that it isn't pulled using sequel query so, that the new dataframe doesn't take up extra memory.

In [ ]:
col_drop = (['Unnamed: 0', "Address", 'Email', 'Email_Update',
                   'Email_Validate', 'Firstname', 'Lastname', 'Mobile','Last updated store external id', 'Last updated store name', 'Registered concept name',
            'Registered store code', 'Registered store external id', 'Registered store name',
            'Registered till name', 'Registered till user name', 'Registered zone name',
            'Slab name', 'User id', 'User identifier', 'isEnabled flag'])

In [ ]:
cap_reg.drop(col_drop, axis=1, inplace=True)

In [ ]:
cap_reg.columns

In [ ]:
cap_reg.info(memory_usage='deep', null_counts=True)
print('_'* 80)

In [ ]:
cap_reg.head()

### Dropping columns with more than 90% missing values.

In [ ]:
length = len(cap_reg)
list = []
for col in cap_reg.columns:
    if (cap_reg[col].count()/length)* 100 <= 10:
        list.append(col)
cap_reg.drop(columns=list, inplace=True)

In [ ]:
list

In [ ]:
cap_reg.describe()

In [ ]:
cap_reg['gender'].unique()

**Data Cleaning of gender feature**   #Using regular expression

---



In [ ]:
#cap_reg['gender'] = cap_reg['gender'].replace(regex={r'^[mM].*$': 'M', r'^[fF].*$': 'F'}) 

In [ ]:
cap_reg.head()

### For features such as nationality, having unique values more than 3 .

In [ ]:
cap_reg['Nationality'].nunique()

### Errors such as OutOfBoundsDatetime coerced 

#### Converted to Day/Month/Year format for ease of viewing. Since, I can't figure out beforehand that the below columns are datetime format: so, these have to be the same. One thing that can be done as precursor is all such columns are appended to a list and a for loop is used with dummy variable col.

---



###OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1406-11-06 20:47:12, Here due to errors='coerce', converts to NaT

In [ ]:
cap_reg['Date of joining'] = pd.to_datetime(cap_reg['Date of joining'], errors='coerce').dt.strftime('%d/%m/%Y')
cap_reg['dateofbirth'] = pd.to_datetime(cap_reg['dateofbirth'] , errors='coerce').dt.strftime('%d/%m/%Y' )
cap_reg['Anniversary'] = pd.to_datetime(cap_reg['Anniversary'], errors='coerce').dt.strftime('%d/%m/%Y')




In [ ]:
cap_reg.head()

In [ ]:
#cap_reg['preferredlanguage'].unique()

### Converting The Languages to English and Arabic

In [ ]:
#cap_reg['preferredlanguage'].unique()

### Will change Other after merging with sales dataframe.

In [ ]:
cap_reg.drop_duplicates(subset='External id', inplace=True)

In [ ]:
final_df = pd.merge(sales1, cap_reg, left_on='custaccount', right_on='External id', how='left').drop(columns='External id')

In [ ]:
final_df.head()

In [ ]:
final_df['churn'].unique()

In [ ]:
(final_df['sales'] == 0).sum()

In [ ]:
final_df.columns

In [ ]:
final_df.head()

In [ ]:
(final_df.groupby('Nationality')['custaccount'].agg('size')).sort_values(ascending=False)

In [ ]:
#final_df['gender'] = final_df['gender'].apply(lambda x : 1 if x== 'F' else ( 0 if x=='M' else np.nan))
#  dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [ ]:
final_df['Anniversary'].unique()

In [ ]:
final_df['Anniversary'] = ((pd.to_datetime('today') - pd.to_datetime(final_df['Anniversary']))/np.timedelta64(1, 'D'))     
final_df['dateofbirth'] = ((pd.to_datetime('today') - pd.to_datetime(final_df['dateofbirth']))/np.timedelta64(1, 'D'))
final_df['Date of joining'] = ((pd.to_datetime('today') - pd.to_datetime(final_df['Date of joining']))/np.timedelta64(1, 'D'))

In [ ]:
final_df.head()

In [ ]:
cols = ['custaccount', 'sales', 'Ladies', 'Home', 'Men', 'Accessories', 'Kids',
       'Lingerie & Nightwear', 'Footwear', 'Beauty', 'Anniversary',
       'Current points', 'Date of joining', 'Lifetime points',
       'Lifetime purchases', 'Nationality', 'dateofbirth', 'gender',
       'preferredlanguage','churn' ]

In [ ]:
final_df = final_df[cols]

In [ ]:
#plt.matshow(final_df.corr())
final_df.corr()

In [ ]:
#final_df['churn'] = final_df['churn'].map({True: 0 , False: 1})

In [ ]:
final_df.head()

In [ ]:
columns = final_df.columns

In [ ]:
final_df['gender'].unique()

#### These three columns Lifetime points, Lifetime purchases & Current points should be input as floating point or int values by the user. In the given data , the columns were input as string data & had to be converted to numeric using pandas.Series.to_numeric. (Pandas.Series.astype could also, have been used.)

In [ ]:
final_df['Lifetime points'] = pd.to_numeric(final_df['Lifetime points'], errors='coerce')
final_df['Lifetime purchases'] = pd.to_numeric(final_df['Lifetime purchases'], errors = 'coerce')
final_df['Current points'] = pd.to_numeric(final_df['Current points'], errors = 'coerce')

#### Separating the different types of columns categorical, continuous .

In [ ]:
categorical = []
continuous = []
list3 = []

#### Dtype should be float64 or object if its numeric , etc then it goes to list3, which means there are no NaNs in  columns in list3 and thus, they do not require imputation.

In [ ]:
for col in columns:
  if final_df[col].dtype.name == 'object':
    categorical.append(col)
  elif final_df[col].dtype.name == 'float64':
    continuous.append(col)
  else:
    list3.append(col)

#### Categorical Value

In [ ]:
categorical

#### Continuous list Value

In [ ]:
continuous

In [ ]:
impute_continuous = []
for col in continuous:
  if final_df[col].isnull().sum() > 0:
    impute_continuous.append(col)

In [ ]:
impute_continuous

In [ ]:
impute_categorical = []
for col in categorical:
  if final_df[col].isnull().sum() > 0:
    impute_categorical.append(col)

In [ ]:
impute_categorical

In [ ]:
# Imputation of continuous features.
for col in impute_continuous:
  iq75, iq25 = np.percentile(final_df[col].dropna(), [75, 25])
  iqr = iq75 - iq25
  final_df[col] = final_df[col].where(final_df[col].notnull(), final_df[col].mean())
  m = (final_df[col] <= iq75 + iqr* 1.5)
  final_df[col] = final_df[col].where(m, iq75 + iqr*1.5)
  m = (final_df[col] >= iq25 - iqr*1.5)
  final_df[col] = final_df[col].where(m, iq25 - iqr*1.5)

In [ ]:
final_df.head()
  

In [ ]:
#final_df['gender'] = final_df['gender'].apply(lambda x : 1 if x== 'F' else ( 0 if x=='M' else np.nan))

In [ ]:
for col in impute_categorical:
  #final_df[col] = final_df[col].astype('str')
  if len(final_df[col].unique()) > 3:
    category1 = (final_df.groupby(col)['custaccount'].agg('size')).idxmax()
    category2 = (final_df.groupby(col)['custaccount'].agg('size')).drop(category1).idxmax()
    m = ((final_df[col] == category1 ) | (final_df[col] == category2))
    final_df[col] = final_df[col].where(~final_df[col].isnull() , category1)
    final_df[col] = final_df[col].where(m , 'Others')  
  else:
    category3 = final_df.groupby(col)['custaccount'].agg('size').idxmax()
    category4 = final_df.groupby(col)['custaccount'].agg('size').drop(category1).idxmax()
    category4 = category4.upper()
    category3 = category3.upper()
    final_df[col] = final_df[col].where(final_df[col].apply(lambda x :type(x)) == type('st'), np.nan)
    final_df[col] = final_df[col].apply(lambda x : category4 if category4[:2] in x.upper() else (category3 if category3[:2] in x.upper() else 'Others'))

In [ ]:
np.nan

In [ ]:
final_df.head()

In [ ]:
final_df['Nationality'].unique()

In [ ]:
final_df['gender'].unique()

In [ ]:
final_df['preferredlanguage'].unique()

In [ ]:
category1, category2

In [ ]:
"""cap_reg['preferredlanguage'] = cap_reg['preferredlanguage'].replace(regex={r'^[Ee][nN].*$': 'English', r'^[Aa][rR].*$': 'Arabic'})
"""ml = final_df.groupby('preferredlanguage')['custaccount'].agg('size').idxmax()
final_df['preferredlanguage'] = final_df['preferredlanguage'].where(~final_df['preferredlanguage'].isnull(), ml)
m = ((cap_reg['preferredlanguage'] == 'English') | (cap_reg['preferredlanguage'] == 'Arabic'))
cap_reg['preferredlanguage'] = cap_reg['preferredlanguage'].where(m, 'Other')""""""

In [ ]:
"""cap_reg['preferredlanguage'].unique()"""

In [ ]:
final_df.head()

In [ ]:
final_df['preferredlanguage'].unique()

### This is Memory information of Colab CPU! 13.35 Gbs of RAM!!!

In [ ]:
!free

### This is CPU processor information , far more processing power!!!

In [ ]:
!cat /proc/cpuinfo

In [ ]:
final_df.dropna(columns='churn')

In [ ]:
final_df.dropna?